In [1]:
# import all necessary libraries
import numpy as np
import pandas as pd
import importlib
import random
from tqdm import tqdm
import datetime
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [4]:
import numpy as np
!pip install scikit-fuzzy
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [5]:
# Set random seed
random.seed(42)
np.random.seed(42)

In [6]:
y_cri = pd.read_pickle('/content/y_cri_df.pkl')

In [7]:
y_cri

,cri
0,0.000000
1,0.000000
2,0.000000
3,0.064588
4,0.000000
...,...
264765,0.192224
264766,0.023216
264767,0.035265
264768,0.072288


In [8]:
enfactor_df = pd.read_pickle('/content/enfactor_array.pickle')

In [9]:
enfactor_df

,Enfactors
0,0.413158
1,0.393421
2,0.413158
3,0.393421
4,0.353947
...,...
264765,0.393421
264766,0.334211
264767,0.334211
264768,0.353947


In [11]:
# Setting up the Fuzzy Inference System to combine the basic CRI (y_cri) with the enfactors
# Define input variables (indices)
enfactors = ctrl.Antecedent(np.arange(0, 1.0001, 0.0001), 'enfactors')
basic_risk_index = ctrl.Antecedent(np.arange(0, 1.0001, 0.0001), 'basic_risk_index')

# Define output variable (ecri_w)
ecri_w = ctrl.Consequent(np.arange(0, 1.0001, 0.0001), 'ecri_w')

# Define membership functions for the input variables

basic_risk_index['vs'] = fuzz.trimf(basic_risk_index.universe, [0, 0, 0.33])
basic_risk_index['s'] = fuzz.trimf(basic_risk_index.universe, [0, 0.33, 0.66])
basic_risk_index['m'] = fuzz.trimf(basic_risk_index.universe, [0.33, 0.66, 1])
basic_risk_index['b'] = fuzz.trimf(basic_risk_index.universe, [0.66, 1, 1])

enfactors['vs'] = fuzz.trimf(enfactors.universe, [0, 0, 0.33])
enfactors['s'] = fuzz.trimf(enfactors.universe, [0, 0.33, 0.66])
enfactors['m'] = fuzz.trimf(enfactors.universe, [0.33, 0.66, 1])
enfactors['b'] = fuzz.trimf(enfactors.universe, [0.66, 1, 1])


# Define membership functions for the output variable
ecri_w['vs'] = fuzz.trimf(ecri_w.universe, [0, 0, 0.33])
ecri_w['s'] = fuzz.trimf(ecri_w.universe, [0, 0.33, 0.66])
ecri_w['m'] = fuzz.trimf(ecri_w.universe, [0.33, 0.66, 1])
ecri_w['b'] = fuzz.trimf(ecri_w.universe, [0.66, 1, 1])


# Define fuzzy rules to combine the indices


#first and second column rules
rule1 = ctrl.Rule(basic_risk_index['vs'] & (enfactors['vs'] | enfactors['s']), ecri_w['vs'])
rule2 = ctrl.Rule(basic_risk_index['s'] & (enfactors['vs'] | enfactors['s']), ecri_w['s'])
rule3 = ctrl.Rule(basic_risk_index['m'] & (enfactors['vs'] | enfactors['s']), ecri_w['m'])
rule4 = ctrl.Rule(basic_risk_index['b'] & (enfactors['vs'] | enfactors['s']), ecri_w['b'])


#third column rules
rule5 = ctrl.Rule((basic_risk_index['vs'] | basic_risk_index['s']) & enfactors['m'], ecri_w['s'])
rule6 = ctrl.Rule(basic_risk_index['m'] & enfactors['m'], ecri_w['m'])
rule7 = ctrl.Rule(basic_risk_index['b'] & enfactors['m'], ecri_w['b'])

#fourth column rules
rule8 = ctrl.Rule(basic_risk_index['vs'] & enfactors['b'], ecri_w['s'])
rule9 = ctrl.Rule(basic_risk_index['s'] & enfactors['b'], ecri_w['m'])
rule10 = ctrl.Rule((basic_risk_index['m'] | basic_risk_index['b']) & enfactors['b'], ecri_w['b'])


# Create a control system
index_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10])

# Create a simulation
index_sim = ctrl.ControlSystemSimulation(index_ctrl)

In [12]:
ecri_w_result = []
for index1, index2 in tqdm(zip(y_cri['cri'], enfactor_df['Enfactors']), total=len(y_cri['cri'])):
  index_sim.input['basic_risk_index'] = index1
  index_sim.input['enfactors'] = index2
  index_sim.compute()
  ecri_w_result.append(index_sim.output['ecri_w'])

100%|██████████| 264770/264770 [55:18<00:00, 79.79it/s]


In [13]:
ecri_w_result

[0.2337809092831982,
 0.2132153172200814,
 0.2337809092831982,
 0.21453676665884835,
 0.15796744511169059,
 0.15796744511169059,
 0.2132153172200814,
 0.21853502826230306,
 0.1542973854876687,
 0.4016543756813015,
 0.15421749815343794,
 0.18444344971343127,
 0.6513721330393251,
 0.7555418591000537,
 0.22020212798533556,
 0.11954646850877715,
 0.2132153172200814,
 0.15796744511169059,
 0.22234682735129935,
 0.26311570045943306,
 0.1719079845436955,
 0.20799327011259705,
 0.267289742076581,
 0.13000310323270944,
 0.2132153172200814,
 0.15796744511169059,
 0.20186000467938484,
 0.2947079455169482,
 0.2132153172200814,
 0.16095876674294943,
 0.15796744511169059,
 0.2132153172200814,
 0.22020545670711997,
 0.12224723516582447,
 0.2869388770962521,
 0.23780660198852707,
 0.11954646850877715,
 0.11954646850877715,
 0.21657054007467527,
 0.15796744511169059,
 0.2337809092831982,
 0.2337809092831982,
 0.11261904753061247,
 0.11023255964081388,
 0.15796744511169059,
 0.5310834752784122,
 0.21321

In [14]:
ecri_w_df = pd.DataFrame(ecri_w_result, columns=["ECRI-W"])

# Save to a pickle file
ecri_w_df.to_pickle('/content/ecriw_w_df.pkl')